# 03 Continuous Integration & Code Coverage

In [1]:
import numpy
import matplotlib.pyplot as plt
%matplotlib inline

/Users/aliceharpole/anaconda3/lib/python3.5/site-packages/matplotlib/font_manager.py:280: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  'Matplotlib is building the font cache using fc-list. '


## Continuous Integration

### Why?
* So you've written a set of tests for your code, you run them and everything passes - great! However, you then go back to work on your code and quickly forget about testing it. Eventually, a few months later after implementing several new features you remember to try testing your code again. You run the set of tests, only to find that they fail 
* Solution: continuous integration.
* This will run your tests for you regularly (e.g. every night, every time you push changes to a repository) and report back to you the results
* Can now spot (almost) instantly when code breaks 

### How?
* Several tools out there: `travis.ci`, `jenkins.ci`, `circle.ci`
* Involve writing a short script which details the computational setup (i.e. any libraries needed) and what code should be run to execute tests

## Code converage
### Why?
* So you have a test suite and you're using continuous integration to run it regularly
* However, how do you know that you are testing all parts of your code? It's all very well to test a few auxilliary functions, but if you're not testing the main part of the code then you still cannot trust the results
* Solution: code coverage
* This will track what parts of the code are being run when tests execute and will highlight areas not currently being tested
* Generally want to aim for > 90% code coverage

### How?
* There exist libraries for most languages that will produce code coverage reports, e.g. `coverage.py` for python, `gcov` for C/C++, `tcov` for C/C++/fortran
* Can use tools like Codecov to integrate these tools with continuous integration, providing an easy-to-use interface to analyse code coverage and keep track of code coverage as develop code
* These tools are also particularly useful if code is written in multiple languages, as will combine reports produced for each of the different languages